## **Example: Solving Quantile Regression via ReHLine**

[![Slides](https://img.shields.io/badge/🦌-ReHLine-blueviolet)](https://rehline-python.readthedocs.io/en/latest/)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LwatjwjnMSB97eLVyuOiUY3sl3A3Ie__?usp=sharing)

In [ ]:
!pip install rehline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.8/238.8 kB 13.0 MB/s eta 0:00:00


In [ ]:
## simulate data
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
import numpy as np

scaler = StandardScaler()

n, d = 10000, 5
X, y = make_regression(n_samples=n, n_features=d, noise=1.0)
X = scaler.fit_transform(X)
y = y/y.std()

In [ ]:
from rehline import ReHLine

qt = [0.25, 0.5, 0.75]

clf = ReHLine(C=1.0/n)
X_fake = clf.make_ReLHLoss(X=X, y=y, loss={'name':'QR', 'qt':qt})
clf.fit(X_fake)

## the first d params are the linear coefficients
## and the last 3 params are the quantile-specific intercept
clf.coef_

array([ 0.59688881,  0.39743301,  0.17675816,  0.45894896,  0.47806579,
       -0.0134518 , -0.00152496,  0.01026677])

In [ ]:
score = [X.dot(clf.coef_[:d]) + clf.coef_[d+l] for l in range(len(qt))]

## report Qs for some samples
X_sample, y_sample = X[:5], y[:5]
q_sample = [X_sample.dot(clf.coef_[:d]) + clf.coef_[d+l] for l in range(len(qt))]

q_sample = np.array(q_sample).T

print('fitted quantiles: %s \n' %qt)
print(q_sample)
print('\n Real Y: \n')
print(y_sample[:,np.newaxis])

fitted quantiles: [0.25, 0.5, 0.75] 

[[-0.20078214 -0.18885531 -0.17706358]
 [-2.48210771 -2.47018088 -2.45838915]
 [-1.31540194 -1.30347511 -1.29168338]
 [-2.70665164 -2.69472481 -2.68293308]
 [ 0.37165816  0.38358499  0.39537672]]

 Real Y: 

[[-0.19786913]
 [-2.51954039]
 [-1.33318935]
 [-2.74956754]
 [ 0.39217369]]
